# Fast QPGA Simulations

We need to ensure the following for fast simulation of QPGA:
1. We aren't building the same matrix for each layer (2 cphase and 1 bs matrix should be sufficient to define all coupling interactions in the QPGA mesh). This saves setup time.
2. Use kronecker phase addition: tf/tf-gpu (and likely torch as well) generally likes the BLAS- and GPU-friendly linear operations over generic kronecker products. Use diag multiplication over dense matmuls when possible.
3. Broadcast kronecker phase addition: variables can be defined for all layers as a single variable. We then use a broadcasting operation to perform kronecker phase addition for all layers at once, which is much faster than looping over the layers.

We can use the above philosophy in designing the QPGA class.

In [1]:
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
import keras

import sys

from keras import backend as K
from keras.models import Sequential
from keras.layers import Layer
from keras.optimizers import SGD, Adam

from qpga import np_to_k_complex, QPGA

K.set_floatx('float64')

Using TensorFlow backend.


## NOON state problem

In [2]:
N = 10
num_samples = 100 

# since the batch is the same training example,
# the gradient is unaffected if batch size is greater than 1
# thus we can set the batch size to be 1 without affecting performance
in_state = np.array([1]  + [0] * (2**N - 1), dtype=np.complex128)
out_state = 1/np.sqrt(2) * np.array([1]  + [0] * (2**N - 2) + [1], dtype=np.complex128)

in_data = np_to_k_complex(np.array([in_state] * num_samples))
out_data = np_to_k_complex(np.array([out_state] * num_samples))

In [3]:
class FrameWriterCallback(keras.callbacks.Callback):
    
    def __init__(self):
        super().__init__()
        self.predict_state = out_data[0:1]
        self.predictions = []
    
    def on_batch_begin(self, batch, logs=None):
        self.predictions.append(self.model.predict(in_data[0:1]))

In [4]:
learning_rate = 0.01
opt = Adam(lr=learning_rate)

model = Sequential([QPGA(N, 4*N)])
model.compile(optimizer=opt, loss='mse', metrics=['mse'])

callback = FrameWriterCallback()

history = model.fit(in_data, out_data, epochs=20, batch_size=1, callbacks=[callback])

Epoch 1/20
100/100 [==============================] - 68s 676ms/step - loss: 5.7336e-04 - mean_squared_error: 5.7336e-04
Epoch 2/20
100/100 [==============================] - 26s 260ms/step - loss: 2.9714e-04 - mean_squared_error: 2.9714e-04
Epoch 3/20
100/100 [==============================] - 26s 261ms/step - loss: 1.8878e-04 - mean_squared_error: 1.8878e-04
Epoch 4/20
100/100 [==============================] - 26s 263ms/step - loss: 1.2813e-04 - mean_squared_error: 1.2813e-04
Epoch 5/20
100/100 [==============================] - 26s 263ms/step - loss: 9.3122e-05 - mean_squared_error: 9.3122e-05
Epoch 6/20
100/100 [==============================] - 26s 263ms/step - loss: 7.0258e-05 - mean_squared_error: 7.0258e-05
Epoch 7/20
100/100 [==============================] - 26s 263ms/step - loss: 5.9780e-05 - mean_squared_error: 5.9780e-05
Epoch 8/20
100/100 [==============================] - 26s 263ms/step - loss: 5.2653e-05 - mean_squared_error: 5.2653e-05
Epoch 9/20
100/100 [============

## Fidelity

In [5]:
from qpga import np_to_complex
np.abs(np.dot(np_to_complex(out_data[0:1].conj())[0], np_to_complex(model.predict(in_data[0:1]))[0]))**2

0.9849644529060256